# Содержание


- [Создание и обучение модели](#3)
    - [Подготовка данных для обучения](#31)
    - [Обучение моделей](#33)
        - [Ансамблевые модели: градиентный бустинг](#334)

# Создание и обучение модели <a id="3"></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("cleaned_dataset.csv", index_col=0)

In [3]:
df.head()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,0,35,170,75,90.0,1.0,1.0,1,1,120,...,126,92,17.1,1,1.0,21,35,40,1,1
1,0,30,180,80,89.0,0.9,1.2,1,1,130,...,148,121,15.8,1,0.9,20,36,27,3,0
2,0,40,165,75,91.0,1.2,1.5,1,1,120,...,74,104,15.8,1,0.9,47,32,68,1,0
3,0,50,175,80,91.0,1.5,1.2,1,1,145,...,104,106,17.6,1,1.1,29,34,18,1,0
4,0,50,165,60,80.0,1.0,1.2,1,1,138,...,117,104,13.8,1,0.8,19,12,25,1,0


На этом шаге происходит обучение модели. Обучение моделей машинного обучения происходит итерационно - пробуются различные модели, перебираются гиперпараметры, сравниваются значения выбранной метрики и выбирается лучшая комбинация.


## Подготовка данных для обучения <a id="31"></a>

Вначале нужно определить, на каких данных будет обучаться модель, а на каких тестироваться. **Традиционный подход** - это разделение исходного набора данных на 3 части (обучение, валидация и тестирование) с пропорции 60/20/20. В данном случае обучающая выборка используется для обучения модели, а валидация и тестирование для получения значения метрики без эффекта переобучения.

Однако существует и другой подход к разбиению данных - разделение на 2 части (обучение и тестирование) по правилу 80-20 (80% тренировочный, 20% тестовый). Зачастую данный метод применяется в тех случаях, когда отсутствует достаточное количество данных как в обучающем, так и в проверочном наборе.  

Перед тем как начать разбивать данные необходимо выделить из исходного набора данных целевую переменную (столбец `DRK_YN`) и сохранить её в отдельную переменную. Ниже приведён код разделения:

In [4]:
x = df.drop(columns=["DRK_YN"], axis=1)
y = df["DRK_YN"]

В ходе выполнения данной работы будет использован подход с разделением исходной выборки на 2 части с пропорцией 80-20, поскольку данный способ является самым популярным способом разбиения данных. Для того, чтобы разбить данные таким образом, существует специальный метод `train_test_split` в библиотеке `scikit-learn`.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

## Обучение модели <a id="33"></a>

### Ансамблевые модели: градиентный бустинг <a id="334"></a>

У каждой модели машинного обучения есть некоторый предел, до которого можно повышать точность, а дальше начинается переобучение и модель уже физически не может предсказать точнее, чем заложено в её природе. Эта природа определяется не только моделью, а связкой модели, т.е. математическим аппаратом и данными, которые эта модель пытается преобразовать. Однако точность работы одной конкретной модели не является пределом при решении задач машинного обучения.

Когда с этим столкнулись первые исследователи они обратились к математической статистике и выяснили, что если взять несколько однородных моделей и среднее их предсказание, но при этом модели обучены на немного разных выборках, то получится, что ошибка среднего равна $\sqrt{n}$, где $n$ это количество моделей. И ошибка выходит меньше за счёт того, что мы берём несколько одинаковых моделей. **Цель** ансамблевых методов - объединить различные классификаторы в метаклассификатор, который обладает лучшей эффективностью обобщения, чем каждый индивидуальный классификатор сам по себе.

В качестве примера ансамблей можно привести феномен "Мудрость толпы". В 1906 году Френсис Гальтон предлагал посетителям ярмарки угадать вес живого быка. Всего в эксперименте приняли участие около 800 человек. Каждый из участников должен был написать свой ответ на карточке и передать её Гальтону. Затем он сложил все ответы и разделил их на общее количество участников, чтобы получить среднее значение. Результат оказался удивительным: среднее значение всех ответов было ближе к истинному весу быка, чем большинство индивидуальных ответов.

Наиболее популярными видами ансамблирования являются:
1. **Бэггинг** (bagging: bootstrap aggregation) - принцип построения композиции, основанный на простом голосовании.
2. **Бустинг** - принцип построения композиции, основанный на последовательном обучении моделей, при котором модели исправляют ошибки друг у друга.
3. **Стекинг** - принцип, при котором происходит комбинация разнородных моделей для построения прогноза.

Вторым видом ансамблирования является **бустинг**. Метод бустинга в чём то схож с методом беггинга: берётся множество одинаковых *слабых* моделей и объединяется, чтобы получить *сильную*. Основное отличие заключается в том, что модели строятся последовательно. Таким образом получается, что каждый последующий алгоритм компенсирует ошибку предыдущих.

Подход бустинга можно сравнить, например, с учениками, решающими домашнее задание. Один ученик делает домашнее задание, но делает с ошибками. Он сверяет с домашним заданием другого ученика и смотрит на свои ошибки и так у всего класса. В конце получается верно выполненное домашнее задание, которое незначительно отличается от идеального.

Исходный алгоритм бустинга можно подытожить в виде следующих основных шагов:
1. Выбрать случайный поднабор обучающих образцов $d_1$ без возвращения из обучающего набора $D$ для обучения слабого ученика $C_1$;
2. Выбрать второй случайный поднабор обучающих образцов $d_2$ без возвращения из обучающего набора и добавить 50% образцов, которые ранее были неправильно классифицированы, для обучения слабого ученика $C_2$;
3. Найти в обучающем наборе $D$ обучающие образцы $d_3$, по которым $C_1$ и $C_2$ расходятся, для обучения слабого ученика $C_3$;
4. Объединить слабых учеников $C_1$, $C_2$ и $C_3$ посредством мажоритарного голосования.

Существует два наиболее распространённых алгоритма бустинга: *адаптивный бустинг* и *градиентный бустинг*.

**Градиентный бустинг**, также как и адаптивный, обучает слабые модели последовательно, исправляя ошибки предыдущих. Результатом градиентного бустинга также является средневзвешенная сумма результатов моделей. Принципиальное отличие от адаптивного бустинга заключается в способе изменения весов. Адаптивный бустинг использует *итеративный метод* оптимизации, в то время как градиентный бустинг использует *градиентный спуск* для оптимизации весов моделей. Это означает, что он ищет оптимальные значения весов, которые минимизируют ошибку модели.

Таким образом, градиентный бустинг - обобщение адаптивного бустинга для дифференцируемых функций.

Перед тем, как перейти к практической реализации данного алгоритма, необходимо импортировать все необходимые модули.

In [6]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Categorical, Real, Integer
import chime
%load_ext chime 

Далее, также как и в предыдущий раз, создадим конвейер из двух составляющих: стандартизатора и самого классификатора.

In [7]:
# Создаем стандартизатор
standardizer = StandardScaler()

# Создаем классификатор
clf = XGBClassifier(random_state=42)

# Создаём конвейер
pipe = Pipeline([("standardizer", standardizer), ("gradient", clf)])

Диапазон тестирования параметров модели ограничен лишь вычислительной мощностью. Поэтому определим набор параметров, которые будут проверятся при кросс-валидации:

In [8]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

params = {
    "gradient__max_depth": Integer(10, 20),
    "gradient__learning_rate": Real(0.001, 1.0, prior='log-uniform'),
    'gradient__subsample': Real(0.0, 1.0),
    'gradient__colsample_bytree': Real(0.5, 1.0),
    'gradient__colsample_bylevel': Real(0.0, 1.0),
    'gradient__colsample_bynode' : Real(0.5, 1.0),
    'gradient__reg_alpha': Real(10.0, 20.0),
    'gradient__reg_lambda': Real(0.0, 10.0),
    'gradient__gamma': Real(0.0, 10.0)
}


Запустим процесс валидации и выведем оптимальные параметры для данного классификатора.

In [9]:
%%time
%%chime
gradient_classifier = BayesSearchCV(
    estimator=pipe, 
    search_spaces=params,
    cv=5, verbose=1, 
    n_jobs=-3, 
    scoring="roc_auc").fit(
    x_train, y_train
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

После завершения этапа обучения выведем наилучшую комбинацию параметров, а также наилучшую оценку, полученную на тренировочных данных.

In [11]:
print(
    f"Оптимальные параметры: {gradient_classifier.best_params_}\nНаилучшая оценка: {gradient_classifier.best_score_:.2%}"
)

Оптимальные параметры: OrderedDict([('gradient__colsample_bylevel', 0.9452050650024018), ('gradient__colsample_bynode', 1.0), ('gradient__colsample_bytree', 1.0), ('gradient__gamma', 5.966638612389175), ('gradient__learning_rate', 0.08893704430415869), ('gradient__max_depth', 17), ('gradient__reg_alpha', 20.0), ('gradient__reg_lambda', 0.0), ('gradient__subsample', 1.0)])
Наилучшая оценка: 82.17%


Оптимальные параметры: OrderedDict([('gradient__colsample_bylevel', 0.9452050650024018), ('gradient__colsample_bynode', 1.0), ('gradient__colsample_bytree', 1.0), ('gradient__gamma', 5.966638612389175), ('gradient__learning_rate', 0.08893704430415869), ('gradient__max_depth', 17), ('gradient__reg_alpha', 20.0), ('gradient__reg_lambda', 0.0), ('gradient__subsample', 1.0)])
Наилучшая оценка: 82.17%